In [1]:
import requests
import time

import pymongo

In [2]:
TOKEN = 'TOKEN'
PLATFORMS = ['BR1', 'EUN1', 'EUW1', 'JP1', 'KR', 'LA1', 'LA2', 'NA1', 'OC1', 'TR1', 'RU']
TIERS = ['CHALLENGER', 'GRANDMASTER', 'MASTER', 'DIAMOND', 'PLATINUM', 'GOLD', 'SILVER', 'BRONZE', 'IRON']
PLAYERS_COLLECTION = list()
PLAYERS_EXCEPTIONS = list()

In [3]:
client = pymongo.MongoClient('localhost', 27017)
db = client['league']
players_db = db['players']

In [4]:
def get_players(platform: str, tier: str) -> None:
    url = f'https://{platform}.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/{tier}/I?page=1'
    
    time.sleep(1.2)
    
    try:
        req = requests.get(url, headers={'X-Riot-Token': TOKEN})
    except Exception as e:
        print('CONNECTION ERROR')
        return

    counter = 0
    players = req.json()

    for player in players:
        counter += 1
        print(f'{counter}/{len(players)}', end='\r')

        summonerName = player['summonerName']
        url = f'https://{platform}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summonerName}'

        time.sleep(1.2)
        
        try:
            accountId = requests.get(url, headers={'X-Riot-Token': TOKEN})
        except Exception as e:
            print('CONNECTION ERROR')
            
        if accountId.status_code == 200:
            accountId = accountId.json()['accountId']

            PLAYERS_COLLECTION.append(
                {
                    'accountId': accountId,
                    'summonerName': summonerName,
                    'tier': tier,
                    'platform': platform
                }
            )

        elif r.status_code == 429:
            print('\nRATE LIMIT EXCEEDED')
            PLAYERS_EXCEPTIONS.append((summonerName, tier, platform))
            time.sleep(8)
        
        elif r.status_code == 404:
            print('\nSUMMONER NOT FOUND')
            ACCOUNT_IDS.remove(accountId)
        
        elif r.status_code == 403:
            print('\nINVALID TOKEN')
            return
        
    print()

In [5]:
def get_player_exceptions() -> None:
    counter = 0
    exceptions = PLAYERS_EXCEPTIONS.copy()
    
    for player in exceptions:
        counter += 1
        print(f'{counter}/{len(exceptions)}', end='\r')

        summonerName = player[0]
        tier = player[1]
        platform = player[2]
        
        url = f'https://{platform}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summonerName}'
        
        time.sleep(1.2)
        
        try:
            accountId = requests.get(url, headers={'X-Riot-Token': TOKEN})
        except Exception as e:            
            print('CONNECTION ERROR')
            PLAYERS_EXCEPTIONS.remove(player)
            continue
            
        if accountId.status_code == 429:
            time.sleep(8)
            continue

        accountId = accountId.json()['accountId']

        PLAYERS_COLLECTION.append(
            {
                'accountId': accountId,
                'summonerName': summonerName,
                'tier': tier,
                'platform': platform
            }
        )
        
        PLAYERS_EXCEPTIONS.remove(player)

In [6]:
for p in PLATFORMS:
    for t in TIERS:
        get_players(p, t)

        while len(PLAYERS_EXCEPTIONS):
            get_player_exceptions()

        players_db.insert_many(PLAYERS_COLLECTION)
        print(f'Added {len(PLAYERS_COLLECTION)} players to the database.')
        
        PLAYERS_COLLECTION = list()